In [1]:
import os

In [28]:
cur_dir = './Training'
def get_file_list(ftype):
    txt_list = []
    for file_name in os.listdir(cur_dir):
        if file_name.endswith(ftype):
            num = file_name.split('.')
            txt_list.append({'file':cur_dir+'/'+file_name, 'value':num[0]})
    return txt_list

def gen_txt_obj(file_name):
    file_obj = []
    begin = 0
    end = 0
    f = open(file_name, 'r', encoding='utf-8')
    lines = f.readlines()
    for line in lines:
        begin = end
        end += len(line)
        file_obj.append({'text':line, 'begin':begin, 'end':end})
    f.close()
    return file_obj

txt_list = get_file_list('.txt')
txt_all = {}
for txt in txt_list:
    temp = gen_txt_obj(txt['file'])
    txt_all[txt['value']] = temp


In [29]:
import re

def gen_ann_obj(file_name):
    f = open(file_name, 'r', encoding='utf8')
    lines = f.readlines()
    entitys = {}
    relations = {}
    for line in lines:
        if line.startswith('T'):
            line = line.strip()
            ##以空格拆分行
            info = re.split('[\t ]', line)
            ##删除info中存在;的元素
            info = [item for item in info if ";" not in item]
            entitys[info[0]] = {'text':info[4] if len(info) == 5 else "\n".join(info[4:len(info)]),'begin':info[2], 'end':info[3]}
        elif line.startswith('R'):
            line = line.strip()
            info = re.split('[\t ]', line)
            relations[info[0]] = {
                'relation':info[1],
                'h':{'name':info[2].split(':')[1]},
                't':{'name':info[3].split(':')[1]}
            }

    for line in lines:
        if line.startswith('*'):
            line = line.strip()
            info = re.split('[\t ]', line)
            coref = info[2:len(info)]
            relation_id = 0
            for i in range(len(coref)):
                for j in range(i+1, len(coref)):
                    if int(entitys[coref[j]]['end']) - int(entitys[coref[i]]['end'])  < 100:
                        relations["*"+str(relation_id)] = {
                            'relation':'coreference',
                            'h':{'name':coref[i]},
                            't':{'name':coref[j]}
                        }
                        relation_id += 1

    relation_list = []
    for item in relations:
        data = relations[item]
        begin_h = entitys[data['h']['name']]['begin']
        begin_t = entitys[data['t']['name']]['begin']
        end_h = entitys[data['h']['name']]['end']
        end_t = entitys[data['t']['name']]['end']
        name_h = entitys[data['h']['name']]['text']
        name_t = entitys[data['t']['name']]['text']
        data['h'] = {'name':name_h, 'begin':int(begin_h), 'end':int(end_h)}
        data['t'] = {'name':name_t, 'begin':int(begin_t), 'end':int(end_t)}
        relation_list.append(relations[item])
    return relation_list

In [30]:
## 处理位置信息
def merge_location(file_index):
    text_data = txt_all[file_index]
    relations = gen_ann_obj(f'{cur_dir}/{file_index}.ann')
    ab_texts = open(f'{cur_dir}/{file_index}.txt', 'r', encoding='utf8')
    ab_texts = "".join(ab_texts.readlines())
    data = []
    for item in relations:
        begin = item['t']['begin'] if item['h']['begin'] > item['t']['begin'] else item['h']['begin']
        end = item['t']['end'] if item['h']['end'] < item['t']['end'] else item['h']['end']
        for i in range(len(text_data)):
            if text_data[i]['begin'] <= begin < text_data[i]['end']:
                text = text_data[i]['text']
                if text_data[i]['end'] < end:
                    text = ab_texts[text_data[i]['begin']:end]
                    # print(ntext+'\n'+text+"\n")
                ## 计算实体在句子中的位置
                item['h']['pos'] = [item['h']['begin'] - text_data[i]['begin'], item['h']['end']-text_data[i]['begin']]
                item['t']['pos'] = [item['t']['begin'] - text_data[i]['begin'], item['t']['end'] - text_data[i]['begin']]
                item['enty'] = [text[item['h']['pos'][0]:item['h']['pos'][1]],
                                text[item['t']['pos'][0]:item['t']['pos'][1]]]
                item['text'] = text
                data.append(item)
    error = 0
    for item in data:
        if item['h']['name'] != item['enty'][0] or item['t']['name'] != item['enty'][1]:
            entity1 = item['h']['name']
            entity1_index = item['text'].find(entity1)
            item['h']['pos'] = [entity1_index, entity1_index+len(entity1)]
            entity2 = item['t']['name']
            entity2_index = item['text'].find(entity2)
            item['t']['pos'] = [entity2_index,entity2_index+len(entity2)]
            item['enty'] = [item['text'][item['h']['pos'][0]:item['h']['pos'][1]],
                                item['text'][item['t']['pos'][0]:item['t']['pos'][1]]]
            error += 1
        # if item['h']['name'] != item['enty'][0] or item['t']['name'] != item['enty'][1]:
        #     data.remove(item)
        del item['enty']
        del item['h']['begin']
        del item['h']['end']
        del item['t']['begin']
        del item['t']['end']
    if error > 1:
        print(f"超过1个错误的文件:{file_index}, 错误个数为{error}, 总数为{len(data)}")
    # print(f"文件{file_index}.ann错误的个数为:{error},总数为{len(data)}")
    return  data, error, len(relations)

# merge_location('3')

In [31]:
import json
from tqdm import tqdm
file_list = get_file_list('.txt')
processed_data = []
error = 0
relations = 0
for one_file in file_list:
    data, err,res = merge_location(one_file['value'])
    processed_data += data
    error += err
    relations += res
print(f"{len(processed_data)}:{relations}, {error}")
over_len = 0
with open('./train.txt','w', encoding='utf8') as fp:
    for item in tqdm(processed_data):
        if len(item['text']) > 128:
            begin = item['h']['pos'][0] if item['h']['pos'][0] < item['t']['pos'][0] else item['t']['pos'][0]
            item['h']['pos'] = [item['h']['pos'][0]-begin, item['h']['pos'][1]-begin]
            item['t']['pos'] = [item['t']['pos'][0]-begin, item['t']['pos'][1]-begin]
            item['text'] = item['text'][begin: len(item['text'])]
        if len(item['text']) < 128:
            fp.write(json.dumps(item, ensure_ascii=False)+'\n')
        else:
            over_len += 1

print(f"over_len:{over_len}")


超过1个错误的文件:1122, 错误个数为19, 总数为30
超过1个错误的文件:293, 错误个数为188, 总数为192
超过1个错误的文件:329, 错误个数为2, 总数为55
超过1个错误的文件:624, 错误个数为2, 总数为11
超过1个错误的文件:669, 错误个数为3, 总数为61
超过1个错误的文件:706, 错误个数为15, 总数为28
超过1个错误的文件:893, 错误个数为47, 总数为94
超过1个错误的文件:898, 错误个数为86, 总数为90
超过1个错误的文件:905, 错误个数为7, 总数为15
超过1个错误的文件:907, 错误个数为32, 总数为35
超过1个错误的文件:993, 错误个数为3, 总数为27
18334:18334, 429


100%|██████████| 18334/18334 [00:00<00:00, 97782.45it/s]

over_len:754


In [21]:
'4'>'14'

True